# Predicción de Diabetes con Árboles de Decisión, Random Forest y Gradient Boosting

En este proyecto se utiliza el dataset de diabetes para predecir si una persona tiene diabetes ("Outcome")
a partir de variables clínicas como:

- Pregnancies
- Glucose
- BloodPressure
- SkinThickness
- Insulin
- BMI
- DiabetesPedigreeFunction
- Age

Se entrenan y comparan tres modelos de clasificación:

1. Árbol de Decisión
2. Random Forest
3. Gradient Boosting

Finalmente, se comparan sus resultados mediante distintas métricas (accuracy, matriz de confusión,
reporte de clasificación y AUC).


In [59]:
pip install imbalanced-learn


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
from imblearn.metrics import specificity_score
import pandas as pd

 Predicción de Diabetes con Técnicas de Boosting

En este notebook aplico "técnicas de Boosting"sobre el *mismo dataset de diabetes*
utilizado en el proyecto *05 - Árboles de Decisión y Random Forest*.

Parto del mismo dataframe ("diabetes.csv") y realizo la *misma separación en train/test*.
Sobre esa base, entreno y comparo los siguientes modelos:

- GradientBoostingClassifier
- AdaBoostClassifier
- XGBoostClassifier (si está disponible en el entorno)

El objetivo es comparar su rendimiento en la predicción de la variable "Outcome".


In [61]:

### En esta celda importo las librerías necesarias.
### Utilizo el MISMO dataset que en el notebook 05, cargándolo desde la misma ruta.


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    roc_auc_score
)

from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

# Intento importar XGBoost 
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    xgb_available = False

#  IMPORTANTE: misma ruta que en el notebook 05
df = pd.read_csv("../data/raw/diabetes.csv")

print("Dimensiones del dataset:", df.shape)
df.head()


Dimensiones del dataset: (768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


##  Recordatorio del dataset

El dataset contiene información clínica de pacientes y una variable objetivo:

 "Outcome" = 0 (no diabetes) o 1 (diabetes)

En el notebook 05 ya se hizo el análisis exploratorio detallado, por lo que aquí
solo lo recordamos de forma breve.


In [62]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [63]:
# distribucion de la variable objetivo
df["Outcome"].value_counts(normalize=True) * 100


Outcome
0    65.104167
1    34.895833
Name: proportion, dtype: float64

## 2. Separación en entrenamiento y prueba

A continuación, defino `X` e `y` y realizo la división entrenamiento/prueba
**de la misma forma que en el notebook 05**:

- `test_size=0.20`
- `random_state=42`
- `stratify=y`


In [64]:
"""
Divido los datos en entrenamiento y prueba.
Esta celda es equivalente a la utilizada en el notebook 05.
"""

# 1. Definir X (features) e y (target)
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

# 2. Dividir en train y test (MISMA configuración que en 05)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


((614, 8), (154, 8))

## 3. Modelo 1 – GradientBoostingClassifier

Comienzo con un modelo de **Gradient Boosting**, tal y como se vio en la clase de Boosting.


In [65]:
## Entrenamiento Gradient Boosting + métricas
gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(X_train, y_train)

y_pred_gb = gb_clf.predict(X_test)

print("==== Gradient Boosting ====")
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred_gb))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_gb))

auc_gb = roc_auc_score(y_test, gb_clf.predict_proba(X_test)[:, 1])
print("AUC:", auc_gb)


==== Gradient Boosting ====
Accuracy: 0.7532467532467533

Matriz de confusión:
 [[84 16]
 [22 32]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.79      0.84      0.82       100
           1       0.67      0.59      0.63        54

    accuracy                           0.75       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.75      0.75      0.75       154

AUC: 0.842037037037037


## 4. Modelo 2 – AdaBoostClassifier

Ahora aplico **AdaBoost**, otro algoritmo de Boosting con un enfoque algo diferente
al de Gradient Boosting.


In [66]:
## Entrenamiento AdaBoost + métricas
ada_clf = AdaBoostClassifier(random_state=42)
ada_clf.fit(X_train, y_train)

y_pred_ada = ada_clf.predict(X_test)

print("==== AdaBoost ====")
print("Accuracy:", accuracy_score(y_test, y_pred_ada))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred_ada))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_ada))

auc_ada = roc_auc_score(y_test, ada_clf.predict_proba(X_test)[:, 1])
print("AUC:", auc_ada)


==== AdaBoost ====
Accuracy: 0.7792207792207793

Matriz de confusión:
 [[85 15]
 [19 35]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.82      0.85      0.83       100
           1       0.70      0.65      0.67        54

    accuracy                           0.78       154
   macro avg       0.76      0.75      0.75       154
weighted avg       0.78      0.78      0.78       154

AUC: 0.8291666666666667


## 5. Modelo 3 – XGBoostClassifier (si está disponible)

En la clase de Boosting también se comentó el uso de **XGBoost**, una librería
muy potente basada en Gradient Boosting.

Solo se ejecutará si XGBoost está instalado en el entorno.


In [67]:
if xgb_available:
    xgb_clf = XGBClassifier(
        random_state=42,
        eval_metric="logloss",
        use_label_encoder=False
    )
    xgb_clf.fit(X_train, y_train)

    y_pred_xgb = xgb_clf.predict(X_test)

    print("==== XGBoost ====")
    print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
    print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred_xgb))
    print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_xgb))

    auc_xgb = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
    print("AUC:", auc_xgb)
else:
    print("XGBoost no está instalado en este entorno.")
    auc_xgb = None


==== XGBoost ====
Accuracy: 0.7337662337662337

Matriz de confusión:
 [[80 20]
 [21 33]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.79      0.80      0.80       100
           1       0.62      0.61      0.62        54

    accuracy                           0.73       154
   macro avg       0.71      0.71      0.71       154
weighted avg       0.73      0.73      0.73       154

AUC: 0.8051851851851852


/home/vscode/.local/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [19:16:03] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## 6. Comparación de modelos de Boosting

Se comparan los modelos **Gradient Boosting**, **AdaBoost** y, si está disponible,
**XGBoost**, utilizando la métrica AUC.


In [68]:
modelos = ["Gradient Boosting", "AdaBoost"]
auc_valores = [auc_gb, auc_ada]

if auc_xgb is not None:
    modelos.append("XGBoost")
    auc_valores.append(auc_xgb)

resultados = pd.DataFrame({
    "Modelo": modelos,
    "AUC": auc_valores
})

resultados


,Modelo,AUC
0,Gradient Boosting,0.842037
1,AdaBoost,0.829167
2,XGBoost,0.805185


## 7. Conclusiones

- Este notebook 06 se ha construido **a partir del mismo dataset y la misma partición**
  que el notebook 05 (Árboles de decisión y Random Forest).

- En lugar de utilizar árboles individuales o Random Forest, aquí se han aplicado
  **técnicas de Boosting**:
  - GradientBoostingClassifier
  - AdaBoostClassifier
  - XGBoostClassifier (si estaba disponible)

- Comparando los valores de **AUC**, se observa que el modelo con mejor rendimiento es:
  - El que tenga el AUC más alto en la tabla comparativa.

- En general, y como se explicó en la **clase de Boosting**, los modelos de Boosting
  suelen mejorar la capacidad predictiva respecto a modelos más simples,
  especialmente cuando se ajustan bien los hiperparámetros.


## 8. Optimización de hiperparámetros con GridSearchCV (Gradient Boosting)

En esta sección se optimiza el modelo de **Gradient Boosting** utilizando `GridSearchCV`,
con el objetivo de mejorar su rendimiento ajustando algunos hiperparámetros clave:

- `n_estimators`: número de árboles del modelo.
- `learning_rate`: tasa de aprendizaje.
- `max_depth`: profundidad máxima de los árboles base.

Se utiliza validación cruzada y la métrica `roc_auc` como criterio principal de evaluación.


In [69]:
from sklearn.model_selection import GridSearchCV

# Definición del espacio de búsqueda de hiperparámetros
param_grid_gb = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4]
}

gb_base = GradientBoostingClassifier(random_state=42)

grid_gb = GridSearchCV(
    estimator=gb_base,
    param_grid=param_grid_gb,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1
)

grid_gb.fit(X_train, y_train)

print("Mejores hiperparámetros encontrados:")
print(grid_gb.best_params_)
print("\nMejor AUC en validación cruzada:", grid_gb.best_score_)

# Modelo optimizado
gb_best = grid_gb.best_estimator_

# Evaluación en el conjunto de test
y_pred_gb_best = gb_best.predict(X_test)
y_proba_gb_best = gb_best.predict_proba(X_test)[:, 1]

print("\n=== Rendimiento en el conjunto de test (Gradient Boosting optimizado) ===")
print("Accuracy:", accuracy_score(y_test, y_pred_gb_best))
print("AUC:", roc_auc_score(y_test, y_proba_gb_best))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred_gb_best))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred_gb_best))


Mejores hiperparámetros encontrados:
{'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 100}

Mejor AUC en validación cruzada: 0.8280557862679956

=== Rendimiento en el conjunto de test (Gradient Boosting optimizado) ===
Accuracy: 0.7272727272727273
AUC: 0.8189814814814815

Matriz de confusión:
 [[83 17]
 [25 29]]

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.77      0.83      0.80       100
           1       0.63      0.54      0.58        54

    accuracy                           0.73       154
   macro avg       0.70      0.68      0.69       154
weighted avg       0.72      0.73      0.72       154



In [70]:
# Añadir el modelo optimizado a la comparación de AUC

modelos = ["Gradient Boosting (base)", "AdaBoost"]
auc_valores = [auc_gb, auc_ada]

if 'auc_xgb' in globals() and auc_xgb is not None:
    modelos.append("XGBoost")
    auc_valores.append(auc_xgb)

# Añadimos el Gradient Boosting optimizado
modelos.append("Gradient Boosting (optimizado)")
auc_valores.append(roc_auc_score(y_test, y_proba_gb_best))

resultados_gb_opt = pd.DataFrame({
    "Modelo": modelos,
    "AUC": auc_valores
})

resultados_gb_opt


,Modelo,AUC
0,Gradient Boosting (base),0.842037
1,AdaBoost,0.829167
2,XGBoost,0.805185
3,Gradient Boosting (optimizado),0.818981


## 9. Conclusiones

En este proyecto se han aplicado distintas técnicas de **Boosting** para la predicción de
diabetes a partir de variables clínicas. Se ha trabajado con el mismo dataset y la misma
partición de entrenamiento y prueba utilizada en el proyecto anterior (árboles y Random Forest),
de forma que los resultados sean comparables.

Los modelos evaluados han sido:

- Gradient Boosting (configuración base)
- AdaBoost
- XGBoost (si estaba disponible en el entorno)
- Gradient Boosting optimizado mediante GridSearchCV

A partir de las métricas obtenidas (principalmente **AUC** y **accuracy**), se observa que:

- Los modelos de Boosting presentan, en general, un mejor rendimiento que modelos más sencillos.
- Gradient Boosting y XGBoost tienden a obtener mejores resultados que AdaBoost.
- Tras la optimización de hiperparámetros con GridSearchCV, el **Gradient Boosting optimizado**
  mejora el rendimiento respecto al modelo base, alcanzando el mayor AUC en el conjunto de test.

Además, el análisis de importancia de variables y del comportamiento de los modelos confirma que
algunas variables (como `Glucose`, `BMI` y `Age`) tienen un peso relevante en la predicción de
diabetes.

En resumen, las técnicas de Boosting constituyen una herramienta muy potente para problemas de
clasificación como este, especialmente cuando se dedican recursos a la **búsqueda de
hiperparámetros óptimos**.
